In [1]:
import base64
import datetime
import time
import json
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from numpy import median
import matplotlib.dates as mdates

np.random.seed(1337)
%matplotlib inline
sns.set(font_scale=1.5)
rcParams['figure.figsize'] = 12, 8
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
# d = pd.read_csv('tmp/clean_part_0_classif2_test.csv')
# for i in range(1, 10):
#     d = pd.concat([d, pd.read_csv('tmp/clean_part_{0}_classif2_test.csv'.format(i))])
# d.to_csv('tmp/clean_classif2_test.csv')
# del d

In [4]:
products = '''ind_ahor_fin_ult1
ind_aval_fin_ult1
ind_cco_fin_ult1
ind_cder_fin_ult1
ind_cno_fin_ult1
ind_ctju_fin_ult1
ind_ctma_fin_ult1
ind_ctop_fin_ult1
ind_ctpp_fin_ult1
ind_deco_fin_ult1
ind_deme_fin_ult1
ind_dela_fin_ult1
ind_ecue_fin_ult1
ind_fond_fin_ult1
ind_hip_fin_ult1
ind_plan_fin_ult1
ind_pres_fin_ult1
ind_reca_fin_ult1
ind_tjcr_fin_ult1
ind_valo_fin_ult1
ind_viv_fin_ult1
ind_nomina_ult1
ind_nom_pens_ult1
ind_recibo_ult1'''.split('\n')

products2 = '''ind_cco_fin_ult1
ind_cno_fin_ult1
ind_dela_fin_ult1
ind_ecue_fin_ult1
ind_reca_fin_ult1
ind_tjcr_fin_ult1
ind_nomina_ult1
ind_nom_pens_ult1
ind_recibo_ult1'''.split('\n')

In [5]:
# d = pd.read_csv('tmp/clean_classif2_test.csv')
# del d['Unnamed: 0']
# d['target_name'] = 'test'
# for product in products:
#     print product
#     tr = pd.read_csv('tmp/clean_sample_100000_classif2_{0}.csv'.format(product))
#     tr['target_name'] = product
#     d = pd.concat([d, tr])
# cols_to_encode = []
# for c in d.columns:
#     if (tr[c].dtype == object and
#     (c not in ['customer_id', 'target', 'target_name'])):
#         cols_to_encode.append(c)
# d = pd.get_dummies(d, columns=cols_to_encode)
# print 'foo'

# d[d.target_name == 'test'].to_csv('tmp/ready_for_prediction_test.csv', index=False)
# print 'bar'

# for product in products:
#     print product
#     d[d.target_name == product].to_csv('tmp/ready_for_prediction_train_{0}.csv'.format(product), index=False)

# del d

In [6]:
for p in products2:
    print '=' * 30
    print 'product:', p
    
    d = pd.read_csv('tmp/ready_for_prediction_train_{0}.csv'.format(p))
    d = d.iloc[np.random.permutation(len(d))]
    del d['customer_id']
    del d['target_name']
    l = d.target.copy()
    del d['target']
    
    cols_to_encode = []
    for c in d.columns:
        if d[c].dtype == object:
            cols_to_encode.append(c)
            
    d = pd.get_dummies(d, columns=cols_to_encode)
    
    print 'value_counts:'
    print l.value_counts()
    
    try:
        lim_train = 3*len(d)/4
        tr = d[:lim_train]
        trl = l[:lim_train]
        te = d[lim_train:]
        tel = l[lim_train:]

        m = XGBClassifier(
                n_estimators=100,
                learning_rate=0.05,
                max_depth=5,
                gamma=0.01,
                scale_pos_weight=15.0)

        m.fit(tr, trl)
        p = m.predict(te)

        print 'roc_auc_score:'
        print roc_auc_score(tel, p)

        cmx = pd.DataFrame(confusion_matrix(tel, p))
        cmx.index.name='actual'
        cmx.columns.name='predicted'
        print 'confusion matrix:'
        print cmx
    except:
        print ':('

product: ind_cco_fin_ult1
value_counts:
0.0    95752
1.0     4248
Name: target, dtype: int64
roc_auc_score:
0.947390715269
confusion matrix:
predicted      0     1
actual                
0          22801  1140
1             61   998
product: ind_cno_fin_ult1
value_counts:
0.0    97323
1.0     2677
Name: target, dtype: int64
roc_auc_score:
0.854497080342
confusion matrix:
predicted      0     1
actual                
0          22463  1861
1            145   531
product: ind_dela_fin_ult1
value_counts:
0.0    98938
1.0     1062
Name: target, dtype: int64
roc_auc_score:
0.73040509057
confusion matrix:
predicted      0    1
actual               
0          24245  478
1            144  133
product: ind_ecue_fin_ult1
value_counts:
0.0    97769
1.0     2231
Name: target, dtype: int64
roc_auc_score:
0.828203641479
confusion matrix:
predicted      0    1
actual               
0          23578  864
1            172  386
product: ind_reca_fin_ult1
value_counts:
0.0    99090
1.0      910
Name: ta

In [8]:
models = {}
for product in products:
    tr = pd.read_csv('tmp/ready_for_prediction_train_{0}.csv'.format(product))
    trl = tr.target.copy()
    
    del tr['customer_id']
    del tr['target']
    del tr['target_name']
    
    m = XGBClassifier(
                n_estimators=100,
                learning_rate=0.05,
                max_depth=5,
                gamma=0.01,
                scale_pos_weight=15.0)
    m.fit(tr, trl)
    models[product] = m
    del tr
    del trl

KeyboardInterrupt: 

In [ ]:
te = pd.read_csv('tmp/ready_for_prediction_test.csv')
del te['target']
del te['target_name']
sub = pd.DataFrame({'customer_id': te.customer_id.copy()})
del te['customer_id']
for product in products:
    sub[product] = models[product].predict_proba(te)[:, 1]
del te

In [ ]:
sub.head(20)

In [ ]:
sub.to_csv('tmp/sub_prob.csv', index=False)

In [ ]:
te = pd.read_csv('tmp/ready_for_prediction_test.csv')
del te['target']
del te['target_name']
sub = pd.DataFrame({'customer_id': te.customer_id.copy()})
del te['customer_id']
for product in products:
    sub[product] = models[product].predict(te)
del te

In [ ]:
sub.head(20)

In [ ]:
sub.to_csv('tmp/sub.csv', index=False)

In [ ]:
prod_prio = sub.sum().sort_values(ascending=True).index.tolist()[:-1]

In [ ]:
prod_prio

In [ ]:
prod_prio[-1]

In [ ]:
s = pd.DataFrame({'ncodpers': sub.customer_id.copy()})
s['added_products'] = prod_prio[-1]

In [ ]:
for p in prod_prio:
    s.loc[sub[p] == 1.0, 'added_products'] = p
s.head(10)

In [ ]:
ss = pd.read_csv('output/sample_submission.csv')
del ss['added_products']
ss.head(10)

In [ ]:
s = pd.merge(ss, s, how='left')
s.head(10)

In [ ]:
s.to_csv('output/s2.csv', index=False)